### Install the required packages

In [ ]:
# %pip install python-dotenv
# %pip install requests
# %pip install openai --user
# %pip install tzdata
# %pip install --upgrade python-certifi-win32
# %pip install --upgrade certifi
# %pip install pip-system-certs

### Load the configuration settings

In [ ]:
from dotenv import dotenv_values,load_dotenv
from openai import AzureOpenAI, AsyncAzureOpenAI
import sys
import os
sys.path.append('../common')
from helper import Helper

api_version = "2024-06-01"

# Load the environment variables
env = os.getenv('GITHUB_ENV') or 'local'
env_name = os.getenv('ENV_NAME') or 'GO-AMER-USE-DV'

load_dotenv()

secrets = dotenv_values("secrets.env")
api_key = secrets.get("API_KEY")
environment = secrets.get("APIM_NAME")
region_override = secrets.get("REGION_OVERRIDE")
default_headers = {
    "Ocp-Apim-Subscription-Key": api_key,
    'x-kpmg-charge-code': '803000015283',
    "x-kpmg-region-override": region_override,
}

helper = Helper(environment, "genai/azure/deployments", api_key, region_override)

helper.log('GITHUB_ENV: ' + env)
helper.log('ENV_NAME: ' + env_name)

In [ ]:
env_config = helper.read_json_config(env_name)

# Check if deployment_info is found
assert env_config is not None, f"Deployment information not found for environment: {env_name}"

chat_model_deployments_list = env_config["api_configuration"]["azure_openai_completions"]['chat_models']

helper.log("Chat Models:")
helper.log(chat_model_deployments_list)

### Embeddings

In [ ]:
embeddings_deployment = env_config["api_configuration"]["azure_openai_completions"]['embeddings_model']

helper.log("Embeddings Models:")
helper.log(embeddings_deployment)

# Create the client
oai_client = AzureOpenAI(
    azure_endpoint=F"https://{environment}/genai/azure",
    api_version=api_version,
    api_key=api_key,
    default_headers = default_headers
)

# Create the embedding request
response = oai_client.embeddings.create(
    input = "Your text string goes here",
    model = embeddings_deployment
)

# Output the response
helper.log(response.data[0].embedding)

# Assert the response is valid
assert len(response.data[0].embedding) > 0

### Chat Completion (Non-Streaming)

In [ ]:
deployment_id = [x for x in chat_model_deployments_list if "gpt-4o" in x][0]

helper.log("Deployment: " + deployment_id)

# Create the client
oai_client = AzureOpenAI(
    azure_endpoint=F"https://{environment}/genai/azure",
    api_version=api_version,
    api_key=api_key,
    default_headers = {
        "Ocp-Apim-Subscription-Key": api_key,
        'x-kpmg-charge-code': '803000015283',
        "x-kpmg-region-override": region_override,
    }
)

# Create the chat completion request
chat_completion = oai_client.chat.completions.create(
    model=deployment_id,
    messages=[
        {"role": "user", "content": "Write me a one paragraph short story"}
    ]
)

# Output the response
helper.log(chat_completion.choices[0].message.content)

# Assert the response
assert len(chat_completion.choices) == 1
assert len(chat_completion.choices[0].message.content) > 0

### Chat Completion (Streaming)

In [ ]:
deployment_id = [x for x in chat_model_deployments_list if "gpt-4o" in x][0]

helper.log("Deployment: " + deployment_id)

api_version = "2024-05-01-preview"

# Create the client
oai_client = AsyncAzureOpenAI(
    azure_endpoint=f"https://{environment}/genai/azure",
    api_version=api_version,
    api_key=api_key,
    default_headers = default_headers
)

# Create the chat completion request
chat_completion = await oai_client.chat.completions.create(
    model=deployment_id,
    messages= [
        { "role": "user", "content": "Write me a two sentence short story." },
    ],
    stream=True
)

parts = []

# Iterate over the streamed response
async for chunk in chat_completion:
    if len(chunk.choices) > 0:
        delta = chunk.choices[0].delta
        if delta.content:
            helper.log(delta.content, '')
            parts.append(delta.content)

# Assert the response
assert len(parts) > 0

In [ ]:
import json
from openai import AzureOpenAI
from datetime import datetime
from zoneinfo import ZoneInfo

# Create the client
oai_client = AzureOpenAI(
    azure_endpoint=F"https://{environment}/genai/azure",
    api_version=api_version,
    api_key=api_key,
    default_headers = default_headers
)

deployment_id = [x for x in chat_model_deployments_list if "gpt-4o" in x][0]

# Simplified timezone data
TIMEZONE_DATA = {
    "tokyo": "Asia/Tokyo",
    "san francisco": "America/Los_Angeles",
    "paris": "Europe/Paris"
}

def get_current_time(location):
    """Get the current time for a given location"""
    location_lower = location.lower()

    for key, timezone in TIMEZONE_DATA.items():
        if key in location_lower:
            print(f"Timezone found for {key}")
            current_time = datetime.now(ZoneInfo(timezone)).strftime("%I:%M %p")
            return json.dumps({
                "location": location,
                "current_time": current_time
            })

    print(f"No timezone data found for {location_lower}")
    return json.dumps({"location": location, "current_time": "unknown"})

# Initial user message
messages = [{"role": "user", "content": "What's the current time in San Francisco"}] # Single function call
#messages = [{"role": "user", "content": "What's the current time in San Francisco, Tokyo, and Paris?"}] # Parallel function call with a single tool/function defined

# Define the function for the model
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "Get the current time in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city name, e.g. San Francisco",
                    },
                },
                "required": ["location"],
            },
        }
    }
]

# First API call: Ask the model to use the function
response = oai_client.chat.completions.create(
    model=deployment_id,
    messages=messages,
    tools=tools,
    tool_choice="auto",
)

# Process the model's response
response_message = response.choices[0].message
messages.append(response_message)

# Handle function calls
if response_message.tool_calls:
    for tool_call in response_message.tool_calls:
        if tool_call.function.name == "get_current_time":
            function_args = json.loads(tool_call.function.arguments)
            helper.log(f"Function arguments: {function_args}")
            time_response = get_current_time(
                location=function_args.get("location")
            )
            messages.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": "get_current_time",
                "content": time_response,
            })
else:
    helper.log("No tool calls were made by the model.")

# Second API call: Get the final response from the model
final_response = oai_client.chat.completions.create(
    model=deployment_id,
    messages=messages,
)

helper.log(final_response.choices[0].message.content)


### Get Deployment IDs List

In [12]:
flattened_chat_model_deployments_list = []
api_version = "2024-10-21"  # Default API version, update as needed
for model in chat_model_deployments_list:
    if "gpt-5" in model:
        flattened_chat_model_deployments_list.append({"api_version": api_version, "model": model})

helper.log("GPT-5 Chat Models:")
helper.log(flattened_chat_model_deployments_list)

GPT-5 Chat Models:
[{'api_version': '2024-10-21', 'model': 'gpt-5-chat-2025-08-07-gs-eus2'}, {'api_version': '2024-10-21', 'model': 'gpt-5-2025-08-07-dzs-eus2'}, {'api_version': '2024-10-21', 'model': 'gpt-5-mini-2025-08-07-dzs-eus2'}, {'api_version': '2024-10-21', 'model': 'gpt-5-nano-2025-08-07-dzs-eus2'}]


### Chat Completion (Non-Streaming)

In [ ]:
import concurrent.futures

def test_deployment(deployment):
    try:
        oai_client = AzureOpenAI(
        azure_endpoint=F"https://{environment}/genai/azure",
        api_version=deployment['api_version'],
        api_key=api_key,
        default_headers = {
            "Ocp-Apim-Subscription-Key": api_key,
            'x-kpmg-charge-code': '803000015283',
            "x-kpmg-region-override": region_override,
        }
    )

        # Create a completion request using GPT-5 model
        response = oai_client.chat.completions.create(
            model=deployment['model'],
            messages=[
                {"role": "user", "content": "Write me a one paragraph short story"}
            ]
        )

        # Output the completion response
        helper.log(response.choices[0].message.content)

        # Validate the response
        assert len(response.choices) == 1
        assert len(response.choices[0].message.content) > 0
        helper.log(f"++ Deployment_ID:{deployment['model']} Succeeded ++")
        helper.log('')
        return True
    except Exception as err:
        try:
            helper.log(f"XX Deployment_ID:{deployment['model']} failed XX, Error: {str(err)}")
        except UnicodeEncodeError:
            helper.log(f"XX Deployment ID failed XX, Error: Unicode encoding error with model name")
        return False

success_count = 0
failure_count = 0

with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(test_deployment, flattened_chat_model_deployments_list))

success_count = results.count(True)
failure_count = results.count(False)

helper.log(f"Total Success: {success_count}, Total Failure: {failure_count} of {len(flattened_chat_model_deployments_list)}")

assert failure_count == 0, f'{failure_count} of {len(flattened_chat_model_deployments_list)} deployments failed'


### Chat Completion (Streaming)

In [ ]:
helper.log('gpt5 streaming')
async def test_deployment_streaming(deployment):
    try:
        oai_client = AsyncAzureOpenAI(
            azure_endpoint=f"https://{environment}/genai/azure",
            api_version=deployment['api_version'],
            api_key=api_key,
            default_headers=default_headers
        )
        response = await oai_client.chat.completions.create(
            model=deployment['model'],
            messages=[{"role": "user", "content": "What is the capital of France?"}],
            stream=True
        )
        parts = []
        async for chunk in response:
            if len(chunk.choices) > 0:
                delta = chunk.choices[0].delta
                if delta.content:
                    helper.log(delta.content, '')
                    parts.append(delta.content)
        helper.log('')
        assert len(parts) > 0
        helper.log(f"++ Deployment_ID:{deployment['model']} Succeeded ++")
        helper.log('')
        return True
    except Exception as err:
        try:
            helper.log(f"XX Deployment_ID:{deployment['model']} failed XX, Error: {str(err)}")
        except UnicodeEncodeError:
            helper.log(f"XX Deployment ID failed XX, Error: Unicode encoding error with model name")
        return False

async def run_streaming_tests_sequentially():
    success_count = 0
    failure_count = 0
    for deployment in flattened_chat_model_deployments_list:
        result = await test_deployment_streaming(deployment)
        if result:
            success_count += 1
        else:
            failure_count += 1
    helper.log(f"Total Success: {success_count}, Total Failure: {failure_count} of {len(flattened_chat_model_deployments_list)}")
    assert failure_count == 0, f'{failure_count} of {len(flattened_chat_model_deployments_list)} deployments failed'

# Run the streaming tests sequentially
await run_streaming_tests_sequentially()

SyntaxError: 'await' outside async function (796667731.py, line 14)